# Data Processing

In [1]:
import pandas as pd
from textdistance import jaro_winkler
import unicodedata

In [2]:
df = pd.read_csv('data/FBref/players_stats_FBref.csv')
print(f"Df shape: {df.shape}")
df.head(5)

Df shape: (9715, 133)


,Player,Nation,Pos,Squad,League,Age,Born,90s,Standard_Gls,Standard_Sh,...,Performance_Crs,Performance_Int,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Aerial_Duels_Won%
0,Max Aarons,eng ENG,DF,Bournemouth,Premier League,23.0,2000.0,13.7,0.0,2.0,...,13.0,8.0,19.0,0.0,1.0,0.0,75.0,5.0,11.0,31.3
1,Joshua Acheampong,eng ENG,DF,Chelsea,Premier League,17.0,2006.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,Tyler Adams,us USA,MF,Bournemouth,Premier League,24.0,1999.0,1.3,0.0,0.0,...,1.0,4.0,3.0,0.0,0.0,0.0,7.0,2.0,3.0,40.0
3,Tosin Adarabioyo,eng ENG,DF,Fulham,Premier League,25.0,1997.0,18.0,2.0,14.0,...,1.0,25.0,11.0,0.0,0.0,0.0,43.0,56.0,28.0,66.7
4,Elijah Adebayo,eng ENG,FW,Luton Town,Premier League,25.0,1998.0,15.8,10.0,31.0,...,5.0,6.0,1.0,0.0,0.0,0.0,34.0,43.0,43.0,50.0


## Basic pre-processing

The first thing we need to do is to exclude goalkeepers from the analysis, as they have different features than outfield players.

In [3]:
df = df[df['Pos'] != 'GK']
print(f"Df shape: {df.shape}")

Df shape: (9025, 133)


The second thing is to keep only "relevant" players, where relevant refers to players that have played a reasonable amount of minutes in the league.

Let's inspect the distribution of the minutes played by the players in the dataset.

In [4]:
df['90s'].describe()

count    9025.000000
mean       12.955224
std        10.452423
min         0.000000
25%         3.200000
50%        11.300000
75%        21.300000
max        46.000000
Name: 90s, dtype: float64

We can observe how 75% of players have played at least 3 matches, and 50% of players have played at more than 10 matches.

A good choice could be to keep only players that have played at least 5 matches in the league.

**Note:** "match" refers to 90 minutes played (not necessarily a full match), so 3 matches correspond to 450 minutes played, which might be enough to grasp some player's performance.

In [5]:
df = df[df['90s'] >= 5]
df.shape

(6203, 133)

<u>**BIG ASSUMPTION:** we are considering data about the 2023/2024 season, and some players might have moved to another team in winter transfer market. Although we could consider some kind of weighted average, we will consider the same player in different teams as different players (not a problem for the theorethical analysis, but consider such preprocessing step for a practical application!).</u>

We now discard some rows which have values not compatible for feature engineering

In [6]:
df = df[df['Standard_Sh'] > 0]

## Feature Engineering

### Shooting 

Our dataframe contains many shooting variables. Notice that we do not actually need all of them, as some are redundant. For example, `Standard_SoT%` is the same as `Standard_SoT` divided by `Standard_Sh`. Similarly, `Standard_G/Sh` is the same as `Standard_Gls` divided by `Standard_Sh`. Moreover, we should rely on a **per 90** basis, as we want to compare players that played different minutes. For some stats the per 90 is already provided, for others we need to calculate it.

Main passages we need to do for shooting:

- `Standard_Gls`            ❌ --> `Standard_Gls_P90` 🆕
- `Standard_Sh`             ❌
- `Standard_SoT`            ❌
- `Standard_SoT%`,          ❌ 
- `Standard_Sh/90`          ✅
- `Standard_SoT/90`         ✅  
- `Standard_G/Sh`           ❌ 
- `Standard_G/SoT`,         ❌ 
- `Standard_Dist`           ✅
- `Standard_FK`             ❌ --> `FK/Sh` 🆕
- `Standard_PK`,            ❌ --> Not interesting
- `Standard_PKatt`,         ❌ --> Not interesting
- `Standard_Pk/Pkatt`,      🆕 I would have Nan for the majority of players, so not interesting for the analysis
- `Expected_xG`,            ❌ --> `Expected_xG_P90` 🆕
- `Expected_npxG`           ❌ --> `Expected_npxG_P90`  🆕
- `Expected_npxG/Sh`        ❌
- `Expected_G-xG`           ❌
- `Expected_np:G-xG`        ❌

In [7]:
SHOOTING_VARS = [
    'Player','Nation','Pos','Squad','League','Age','Born','90s',	
    'Standard_Gls', 'Standard_Sh', 'Standard_SoT','Standard_SoT%',
    'Standard_Sh/90', 'Standard_SoT/90', 'Standard_G/Sh','Standard_G/SoT',
    'Standard_Dist', 'Standard_FK', 'Standard_PK','Standard_PKatt', 
    'Expected_xG', 'Expected_npxG', 'Expected_npxG/Sh',
    'Expected_G-xG', 'Expected_np:G-xG']

In [8]:
shooting_df = df[SHOOTING_VARS].copy()
shooting_df['Standard_Gls/90'] = shooting_df['Standard_Gls'] / shooting_df['90s']
shooting_df['Expected_xG/90'] = shooting_df['Expected_xG'] / shooting_df['90s']
shooting_df['Expected_npxG/90'] = shooting_df['Expected_npxG'] / shooting_df['90s']
shooting_df['Expected_npxG/90'] = shooting_df['Expected_npxG'] / shooting_df['90s']
shooting_df['Standard_FK/Sh'] = shooting_df['Standard_FK'] / shooting_df['Standard_Sh']

shooting_df = shooting_df.drop(columns=[
    'Standard_Gls', 'Standard_Sh', 'Standard_SoT','Standard_SoT%','Standard_G/Sh','Standard_G/SoT', 'Standard_FK', 'Standard_PK', 'Standard_PKatt', 'Expected_xG', 'Expected_npxG', 'Expected_npxG/Sh', 'Expected_G-xG', 'Expected_np:G-xG'])

In [9]:
shooting_df.shape

(6158, 15)

### Passing

Following the same logic explained above, we need to do:

- `Total_Cmp` ❌ --> `Total_Cmp_P90` 🆕
- `Total_Att`  ❌ -->`Total_Att_P90`  🆕
- `Total_Cmp%`❌
- `Total_TotDist`❌ --> `Total_TotDist / Total_Cmp`🆕
- `Total_PrgDist`❌ --> `Total_PrgDist / Total_Cmp`🆕
- `Short_Cmp` ❌ --> `Short_Cmp_P90` 🆕
- `Short_Att` ❌ --> `Short_Att_P90` 🆕
- `Short_Cmp%` ❌
- `Medium_Cmp` ❌ --> `Medium_Cmp_P90` 🆕
- `Medium_Att` ❌ --> `Medium_Att_P90` 🆕
- `Medium_Cmp%` ❌
- `Long_Cmp` ❌ --> `Long_Cmp_P90` 🆕
- `Long_Att` ❌ --> `Long_Att_P90` 🆕
- `Long_Cmp%` ❌
- `Ast` ❌ --> `Ast_P90` 🆕
- `xAG` ❌ --> `xAG_P90` 🆕
- `Expected_xA` ❌ --> `Expected_xA_P90` 🆕
- `Expected_A-xAG` ✅ does not even need to be converted into P90 in my opinion
- `KP` ❌ --> `KP_P90` 🆕
- `1/3` ❌ --> `1/3_P90` 🆕
- `PPA` ❌ --> `PPA_P90` 🆕
- `CrsPA` ❌ --> `CrsPA_P90` 🆕
- `PrgP` ❌ --> `PrgP_P90` 🆕


Let's see if there are players with 0 passes (pretty unlikely, but let's check it out).

In [10]:
print(f"Players with 0 Attempted passages: are {(df['Total_Att'] == 0).sum()}")

Players with 0 Attempted passages: are 0


In [11]:
PASSING_VARS = [
    'Player','Nation','Pos','Squad','League','Age','Born','90s',
    'Total_Cmp', 'Total_Att', 'Total_Cmp%', 'Total_TotDist',
    'Total_PrgDist', 'Short_Cmp', 'Short_Att', 'Short_Cmp%', 'Medium_Cmp',
    'Medium_Att', 'Medium_Cmp%', 'Long_Cmp', 'Long_Att', 'Long_Cmp%', 'Ast',
    'xAG', 'Expected_xA', 'Expected_A-xAG', 'KP', '1/3', 'PPA', 'CrsPA','PrgP']

In [12]:
passing_df = df[PASSING_VARS].copy()

passing_df['Total_Cmp/90'] = passing_df['Total_Cmp'] / passing_df['90s']
passing_df['Total_Att/90'] = passing_df['Total_Att'] / passing_df['90s']
passing_df['Average_TotDist'] = passing_df['Total_TotDist'] / passing_df['Total_Cmp']
passing_df['Average_PrgDist'] = passing_df['Total_PrgDist'] / passing_df['Total_Cmp']
passing_df['Short_Cmp/90'] = passing_df['Short_Cmp'] / passing_df['90s']
passing_df['Short_Att/90'] = passing_df['Short_Att'] / passing_df['90s']
passing_df['Medium_Cmp/90'] = passing_df['Medium_Cmp'] / passing_df['90s']
passing_df['Medium_Att/90'] = passing_df['Medium_Att'] / passing_df['90s']
passing_df['Long_Cmp/90'] = passing_df['Long_Cmp'] / passing_df['90s']
passing_df['Long_Att/90'] = passing_df['Long_Att'] / passing_df['90s']
passing_df['Ast/90'] = passing_df['Ast'] / passing_df['90s']
passing_df['xAG/90'] = passing_df['xAG'] / passing_df['90s']
passing_df['Expected_xA/90'] = passing_df['Expected_xA'] / passing_df['90s']
passing_df['KP/90'] = passing_df['KP'] / passing_df['90s']
passing_df['1/3/90'] = passing_df['1/3'] / passing_df['90s']
passing_df['PPA/90'] = passing_df['PPA'] / passing_df['90s']
passing_df['CrsPA/90'] = passing_df['CrsPA'] / passing_df['90s']
passing_df['PrgP/90'] = passing_df['PrgP'] / passing_df['90s']


passing_df.drop(columns=['Total_Cmp', 'Total_Att', 'Total_TotDist', 'Total_PrgDist', 
                         'Short_Cmp', 'Short_Att', 'Short_Cmp%', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp%', 
                         'Long_Cmp', 'Long_Att', 'Long_Cmp%', 'Ast', 
                         'xAG', 'Expected_xA', 'KP', '1/3', 'PPA', 'CrsPA', 'PrgP'], inplace=True)

In [13]:
passing_df.columns

Index(['Player', 'Nation', 'Pos', 'Squad', 'League', 'Age', 'Born', '90s',
       'Total_Cmp%', 'Expected_A-xAG', 'Total_Cmp/90', 'Total_Att/90',
       'Average_TotDist', 'Average_PrgDist', 'Short_Cmp/90', 'Short_Att/90',
       'Medium_Cmp/90', 'Medium_Att/90', 'Long_Cmp/90', 'Long_Att/90',
       'Ast/90', 'xAG/90', 'Expected_xA/90', 'KP/90', '1/3/90', 'PPA/90',
       'CrsPA/90', 'PrgP/90'],
      dtype='object')

### Pass types

- `Pass_Types_Live` ❌ --> `Pass_Types_Live_P90` 🆕
- `Pass_Types_Dead` ❌ --> `Pass_Types_Dead_P90` 🆕
- `Pass_Types_FK` ❌ --> `Pass_Types_FK_P90` 🆕
- `Pass_Types_TB` ❌ --> `Pass_Types_TB_P90` 🆕
- `Pass_Types_Sw` ❌ --> `Pass_Types_Sw_P90` 🆕
- `Pass_Types_Crs` ❌ --> `Pass_Types_Crs_P90` 🆕
- `Pass_Types_TI` ❌ --> `Pass_Types_TI_P90` 🆕
- `Pass_Types_CK` ❌ --> Not interesting in our opinion
- `Corner_Kicks_In` ❌ --> Not interesting in our opinion
- `Corner_Kicks_Out` ❌ --> Not interesting in our opinion 
- `Corner_Kicks_Str` ❌ --> Not interesting in our opinion
- `Outcomes_Cmp` ❌ --> `Outcomes_Cmp_P90` 🆕
- `Outcomes_Off` ❌ --> `Outcomes_Off_P90` 🆕
- `Outcomes_Blocks`--> `Outcomes_Blocks_P90` 🆕

In [14]:
PASS_TYPES_VARS = [
    'Player','Nation','Pos','Squad','League','Age','Born','90s',
    'Pass_Types_Live', 'Pass_Types_Dead', 'Pass_Types_FK', 'Pass_Types_TB',
    'Pass_Types_Sw', 'Pass_Types_Crs', 'Pass_Types_CK', 'Pass_Types_TI',
    'Corner_Kicks_In', 'Corner_Kicks_Out', 'Corner_Kicks_Str',
    'Outcomes_Cmp', 'Outcomes_Off', 'Outcomes_Blocks'
    ]

In [15]:
pass_types_df = df[PASS_TYPES_VARS].copy()

pass_types_df['Pass_Types_Live/90'] = pass_types_df['Pass_Types_Live'] / pass_types_df['90s']
pass_types_df['Pass_Types_Dead/90'] = pass_types_df['Pass_Types_Dead'] / pass_types_df['90s']
pass_types_df['Pass_Types_FK/90'] = pass_types_df['Pass_Types_FK'] / pass_types_df['90s']
pass_types_df['Pass_Types_TB/90'] = pass_types_df['Pass_Types_TB'] / pass_types_df['90s']
pass_types_df['Pass_Types_Sw/90'] = pass_types_df['Pass_Types_Sw'] / pass_types_df['90s']
pass_types_df['Pass_Types_Crs/90'] = pass_types_df['Pass_Types_Crs'] / pass_types_df['90s']
pass_types_df['Pass_Types_TI/90'] = pass_types_df['Pass_Types_TI'] / pass_types_df['90s']
pass_types_df['Outcomes_Cmp/90'] = pass_types_df['Outcomes_Cmp'] / pass_types_df['90s']
pass_types_df['Outcomes_Off/Cmp'] = pass_types_df['Outcomes_Off'] / pass_types_df['Outcomes_Cmp']
pass_types_df['Outcomes_Blocks/Cmp'] = pass_types_df['Outcomes_Blocks'] / pass_types_df['Outcomes_Cmp']


pass_types_df.drop(columns=['Pass_Types_Live', 'Pass_Types_Dead', 'Pass_Types_FK', 'Pass_Types_TB', 
                            'Pass_Types_Sw', 'Pass_Types_Crs', 'Pass_Types_TI', 'Pass_Types_CK', 
                            'Outcomes_Cmp', 'Outcomes_Off', 'Outcomes_Blocks',
                            'Corner_Kicks_In','Corner_Kicks_Out','Corner_Kicks_Str'], inplace=True)

In [16]:
pass_types_df.shape

(6158, 18)

### Goal and shot creation

- `SCA_SCA` ❌ 
- `SCA_SCA90`'✅
- `SCA_Types_PassLive` ❌ --> `SCA_Types_PassLive_P90` 🆕
- `SCA_Types_PassDead` ❌ --> `SCA_Types_PassDead_P90` 🆕
- `SCA_Types_TO` ❌ --> `SCA_Types_TO_P90` 🆕
- `SCA_Types_Sh` ❌ --> `SCA_Types_Sh_P90` 🆕
- `SCA_Types_Fld` ❌ --> `SCA_Types_Fld_P90` 🆕
- `SCA_Types_Def` ❌ --> `SCA_Types_Def_P90` 🆕
- `GCA_GCA` ❌ 
- `GCA_GCA90` ✅
- `GCA_Types_PassLive` ❌ --> `GCA_Types_PassLive%` 🆕
- `GCA_Types_PassDead` ❌ --> `GCA_Types_PassDead%` 🆕
- `GCA_Types_TO` ❌ --> `GCA_Types_TO_P90` 🆕
- `GCA_Types_Sh` ❌ --> `GCA_Types_Sh_P90` 🆕
- `GCA_Types_Fld` ❌ --> `GCA_Types_Fld_P90` 🆕
- `GCA_Types_Def` ❌ --> `GCA_Types_Def_P90` 🆕

In [17]:
GCA_VARS = [
    'Player','Nation','Pos','Squad','League','Age','Born','90s',
    'SCA_SCA', 'SCA_SCA90', 'SCA_Types_PassLive', 'SCA_Types_PassDead',
    'SCA_Types_TO', 'SCA_Types_Sh', 'SCA_Types_Fld', 'SCA_Types_Def',
    'GCA_GCA', 'GCA_GCA90', 'GCA_Types_PassLive', 'GCA_Types_PassDead',
    'GCA_Types_TO', 'GCA_Types_Sh', 'GCA_Types_Fld', 'GCA_Types_Def'
    ]

In [18]:
gca_df = df[GCA_VARS].copy()

gca_df['SCA_Types_PassLive/90'] = gca_df['SCA_Types_PassLive'] / gca_df['90s']
gca_df['SCA_Types_PassDead/90'] = gca_df['SCA_Types_PassDead'] / gca_df['90s']
gca_df['SCA_Types_TO/90'] = gca_df['SCA_Types_TO'] / gca_df['90s']
gca_df['SCA_Types_Sh/90'] = gca_df['SCA_Types_Sh'] / gca_df['90s']
gca_df['SCA_Types_Fld/90'] = gca_df['SCA_Types_Fld'] / gca_df['90s']
gca_df['SCA_Types_Def/90'] = gca_df['SCA_Types_Def'] / gca_df['90s']
gca_df['GCA_Types_PassLive/90'] = gca_df['GCA_Types_PassLive'] / gca_df['90s']
gca_df['GCA_Types_PassDead/90'] = gca_df['GCA_Types_PassDead'] / gca_df['90s']
gca_df['GCA_Types_TO/90'] = gca_df['GCA_Types_TO'] / gca_df['90s']
gca_df['GCA_Types_Sh/90'] = gca_df['GCA_Types_Sh'] / gca_df['90s']
gca_df['GCA_Types_Fld/90'] = gca_df['GCA_Types_Fld'] / gca_df['90s']
gca_df['GCA_Types_Def/90'] = gca_df['GCA_Types_Def'] / gca_df['90s']


gca_df.drop(columns=['SCA_SCA', 'SCA_Types_PassLive', 'SCA_Types_PassDead', 'SCA_Types_TO', 'SCA_Types_Sh', 'SCA_Types_Fld', 'SCA_Types_Def', 
                    'GCA_GCA', 'GCA_Types_PassLive', 'GCA_Types_PassDead', 'GCA_Types_TO', 'GCA_Types_Sh', 'GCA_Types_Fld', 'GCA_Types_Def'], inplace=True)


In [19]:
gca_df.shape

(6158, 22)

### Defensive Actions

- `Tackles_Tkl` ❌ --> `Tackles_Tkl_P90` 🆕
- `Tackles_TklW` ❌ --> `Tackles_TklW_P90` 🆕
- `Tackles_Def_3rd` ❌ --> `Tackles_Def_3rd_P90` 🆕
- `Tackles_Mid_3rd` ❌ --> `Tackles_Mid_3rd_P90` 🆕
- `Tackles_Att_3rd` ❌ --> `Tackles_Att_3rd_P90` 🆕
- `Challenges_Tkl` ❌ --> `Challenges_Tkl_P90` 🆕
- `Challenges_Att` ❌ --> `Challenges_Att_P90` 🆕
- `Challenges_Tkl%` ❌ not available for all players
- `Challenges_Lost` ❌ --> `Challenges_Lost_P90` 🆕
- `Blocks_Blocks` ❌ --> `Blocks_Blocks_P90` 🆕
- `Blocks_Sh` ❌ --> `Blocks_Sh_P90` 🆕
- `Blocks_Pass` ❌ --> `Blocks_Pass_P90` 🆕
- `Int` ❌ --> `Int_P90` 🆕
- `Tkl+Int` ❌ --> Not interesting
- `Clr` ❌ --> `Clr_P90` 🆕
- `Err` ❌ --> `Err_P90` 🆕

In [20]:
DEFENSE_VARS = [
    'Player','Nation','Pos','Squad','League','Age','Born','90s',
    'Tackles_Tkl', 'Tackles_TklW', 'Tackles_Def_3rd', 'Tackles_Mid_3rd',
    'Tackles_Att_3rd', 'Challenges_Tkl', 'Challenges_Att',
    'Challenges_Tkl%', 'Challenges_Lost', 'Blocks_Blocks', 'Blocks_Sh',
    'Blocks_Pass', 'Int', 'Tkl+Int', 'Clr', 'Err'
    ]

In [21]:
defense_df = df[DEFENSE_VARS].copy()

defense_df['Tackles_Tkl/90']     = defense_df['Tackles_Tkl'] / defense_df['90s']
defense_df['Tackles_TklW/90']    = defense_df['Tackles_TklW'] / defense_df['90s']
defense_df['Tackles_Def_3rd/90'] = defense_df['Tackles_Def_3rd'] / defense_df['90s']
defense_df['Tackles_Mid_3rd/90'] = defense_df['Tackles_Mid_3rd'] / defense_df['90s']
defense_df['Tackles_Att_3rd/90'] = defense_df['Tackles_Att_3rd'] / defense_df['90s']
defense_df['Challenges_Tkl/90']  = defense_df['Challenges_Tkl'] / defense_df['90s']
defense_df['Challenges_Att/90']  = defense_df['Challenges_Att'] / defense_df['90s']
defense_df['Challenges_Lost/90']    = defense_df['Challenges_Lost'] / defense_df['Challenges_Att']
defense_df['Blocks_Blocks/90']   = defense_df['Blocks_Blocks'] / defense_df['90s']
defense_df['Blocks_Sh/90']       = defense_df['Blocks_Sh'] / defense_df['90s']
defense_df['Blocks_Pass/90']     = defense_df['Blocks_Pass'] / defense_df['90s']
defense_df['Int/90']             = defense_df['Int'] / defense_df['90s']
defense_df['Clr/90']             = defense_df['Clr'] / defense_df['90s']
defense_df['Err/90']             = defense_df['Err'] / defense_df['90s']


defense_df = defense_df.drop(columns=['Tackles_Tkl', 'Tackles_TklW', 'Tackles_Def_3rd', 'Tackles_Mid_3rd', 'Tackles_Att_3rd', 
                                      'Challenges_Tkl', 'Challenges_Tkl%', 'Challenges_Att', 'Challenges_Lost', 
                                      'Blocks_Blocks', 'Blocks_Sh', 'Blocks_Pass', 
                                      'Tkl+Int', 'Int', 'Clr', 'Err'])

In [22]:
defense_df.columns

Index(['Player', 'Nation', 'Pos', 'Squad', 'League', 'Age', 'Born', '90s',
       'Tackles_Tkl/90', 'Tackles_TklW/90', 'Tackles_Def_3rd/90',
       'Tackles_Mid_3rd/90', 'Tackles_Att_3rd/90', 'Challenges_Tkl/90',
       'Challenges_Att/90', 'Challenges_Lost/90', 'Blocks_Blocks/90',
       'Blocks_Sh/90', 'Blocks_Pass/90', 'Int/90', 'Clr/90', 'Err/90'],
      dtype='object')

### Possession

- `Touches_Touches` ❌ --> `Touches_Touches_P90` 🆕
- `Touches_Def_Pen` ❌ --> `Touches_Def_Pen_P90` 🆕
- `Touches_Def_3rd` ❌ --> `Touches_Def_3rd_P90` 🆕
- `Touches_Mid_3rd` ❌ --> `Touches_Mid_3rd_P90` 🆕
- `Touches_Att_3rd` ❌ --> `Touches_Att_3rd_P90` 🆕
- `Touches_Att_Pen` ❌ --> `Touches_Att_Pen_P90` 🆕
- `Touches_Live` ❌ --> Equal to `Touches_Touches`
- `Take-Ons_Att` ❌ --> `Take-Ons_Att_P90` 🆕
- `Take-Ons_Succ` ❌ --> `Take-Ons_Succ_P90` 🆕
- `Take-Ons_Succ%` ❌ not available for all players
- `Take-Ons_Tkld` ❌ --> `Take-Ons_Tkld_P90` 🆕
- `Take-Ons_Tkld%` ❌ not available for all players
- `Carries_Carries` ❌ --> `Carries_Carries_P90` 🆕
- `Carries_TotDist` ❌ --> `Carries_AverageTotDist` 🆕
- `Carries_PrgDist` ❌ --> `Carries_AveragePrgDist` 🆕
- `Carries_PrgC` ❌ --> `Carries_PrgC_P90` 🆕
- `Carries_1/3` ❌ --> `Carries_1/3_P90` 🆕
- `Carries_CPA` ❌ --> `Carries_CPA_P90` 🆕
- `Carries_Mis` ❌ --> `Carries_Mis_P90` 🆕
- `Carries_Dis` ❌ --> `Carries_Dis_P90` 🆕
- `Receiving_Rec` ❌ --> `Receiving_Rec_P90` 🆕
- `Receiving_PrgR` ❌ --> `Receiving_PrgR_P90` 🆕

In [23]:
POSSESSION_VARS = [
    'Player','Nation','Pos','Squad','League','Age','Born','90s',
    'Touches_Touches', 'Touches_Def_Pen', 'Touches_Def_3rd',
    'Touches_Mid_3rd', 'Touches_Att_3rd', 'Touches_Att_Pen', 'Touches_Live',
    'Take-Ons_Att', 'Take-Ons_Succ', 'Take-Ons_Succ%', 'Take-Ons_Tkld',
    'Take-Ons_Tkld%', 'Carries_Carries', 'Carries_TotDist',
    'Carries_PrgDist', 'Carries_PrgC', 'Carries_1/3', 'Carries_CPA',
    'Carries_Mis', 'Carries_Dis', 'Receiving_Rec', 'Receiving_PrgR'
    ]

In [24]:
possession_df = df[POSSESSION_VARS].copy()

possession_df['Touches_Touches/90'] = possession_df['Touches_Touches'] / possession_df['90s']
possession_df['Touches_Def_Pen/90'] = possession_df['Touches_Def_Pen'] / possession_df['90s']
possession_df['Touches_Def_3rd/90'] = possession_df['Touches_Def_3rd'] / possession_df['90s']
possession_df['Touches_Mid_3rd/90'] = possession_df['Touches_Mid_3rd'] / possession_df['90s']
possession_df['Touches_Att_3rd/90'] = possession_df['Touches_Att_3rd'] / possession_df['90s']
possession_df['Touches_Att_Pen/90'] = possession_df['Touches_Att_Pen'] / possession_df['90s']
possession_df['Take-Ons_Att/90'] = possession_df['Take-Ons_Att'] / possession_df['90s']
possession_df['Take-Ons_Succ/90'] = possession_df['Take-Ons_Succ'] / possession_df['90s']
possession_df['Take-Ons_Tkld/90'] = possession_df['Take-Ons_Tkld'] / possession_df['90s']
possession_df['Carries_Carries/90'] = possession_df['Carries_Carries'] / possession_df['90s']
possession_df['Carries_AverageTotDist'] = possession_df['Carries_TotDist'] / possession_df['Carries_Carries']
possession_df['Carries_AveragePrgDist'] = possession_df['Carries_PrgDist'] / possession_df['Carries_Carries']
possession_df['Carries_PrgC/90'] = possession_df['Carries_PrgC'] / possession_df['90s']
possession_df['Carries_1/3/90'] = possession_df['Carries_1/3'] / possession_df['90s']
possession_df['Carries_CPA/90'] = possession_df['Carries_CPA'] / possession_df['90s']
possession_df['Carries_Mis/90'] = possession_df['Carries_Mis'] / possession_df['90s']
possession_df['Carries_Dis/90'] = possession_df['Carries_Dis'] / possession_df['90s']
possession_df['Receiving_Rec/90'] = possession_df['Receiving_Rec'] / possession_df['90s']
possession_df['Receiving_PrgR/90'] = possession_df['Receiving_PrgR'] / possession_df['90s']

possession_df.drop(columns=['Touches_Touches', 'Touches_Def_Pen', 'Touches_Def_3rd',
    'Touches_Mid_3rd', 'Touches_Att_3rd', 'Touches_Att_Pen', 'Touches_Live',
    'Take-Ons_Att','Take-Ons_Succ','Take-Ons_Succ%','Take-Ons_Tkld', 'Take-Ons_Tkld%',
    'Carries_Carries', 'Carries_TotDist','Carries_PrgDist', 'Carries_PrgC', 'Carries_1/3', 
    'Carries_CPA','Carries_Mis', 'Carries_Dis', 'Receiving_Rec', 'Receiving_PrgR'], inplace=True)

### Miscellaneous Stats

- `Performance_CrdY` ❌ --> `Performance_CrdY_P90` 🆕
- `Performance_CrdR` ❌ Not interesting in our opinion
- `Performance_2CrdY` ❌ Not interesting in our opinion
- `Performance_Fls` ❌ --> `Performance_Fls_P90` 🆕
- `Performance_Fld` ❌ --> `Performance_Fld_P90` 🆕
- `Performance_Off` ❌ --> `Performance_Off_P90` 🆕
- `Performance_Crs` ❌ --> `Performance_Crs_P90` 🆕
- `Performance_Int` ❌ --> `Performance_Int_P90` 🆕
- `Performance_TklW` ❌ Already considered in defensive actions
- `Performance_PKcon` ❌ Not interesting in our opinion
- `Performance_OG` ❌ Not interesting in our opinion
- `Performance_Recov` ❌ --> `Performance_Recov_P90` 🆕
- `Aerial_Duels_Won` ❌ --> `Aerial_Duels_Won_P90` 🆕
- `Aerial_Duels_Lost` ❌ --> `Aerial_Duels_Lost_P90` 🆕
- `Aerial_Duels_Won%` ❌ Not interesting in our opinion

In [25]:
MISC_VARS = [
    'Player','Nation','Pos','Squad','League','Age','Born','90s',
    'Performance_CrdY','Performance_CrdR', 'Performance_2CrdY', 'Performance_Fls',
    'Performance_Fld', 'Performance_Off', 'Performance_Crs',
    'Performance_Int', 'Performance_TklW', 'Performance_PKwon',
    'Performance_PKcon', 'Performance_OG', 'Performance_Recov',
    'Aerial_Duels_Won', 'Aerial_Duels_Lost', 'Aerial_Duels_Won%'
    ]

In [26]:
misc_df = df[MISC_VARS].copy()

misc_df['Performance_CrdY/90'] = misc_df['Performance_CrdY'] / misc_df['90s']
misc_df['Performance_Fls/90'] = misc_df['Performance_Fls'] / misc_df['90s']
misc_df['Performance_Fld/90'] = misc_df['Performance_Fld'] / misc_df['90s']
misc_df['Performance_Off/90'] = misc_df['Performance_Off'] / misc_df['90s']
misc_df['Performance_Crs/90'] = misc_df['Performance_Crs'] / misc_df['90s']
misc_df['Performance_Int/90'] = misc_df['Performance_Int'] / misc_df['90s']
misc_df['Aerial_Duels_Won/90'] = misc_df['Aerial_Duels_Won'] / misc_df['90s']
misc_df['Aerial_Duels_Lost/90'] = misc_df['Aerial_Duels_Lost'] / misc_df['90s']

misc_df.drop(
    columns=[
        'Performance_CrdY','Performance_CrdR', 'Performance_2CrdY', 'Performance_Fls',
        'Performance_Fld', 'Performance_Off', 'Performance_Crs',
        'Performance_Int', 'Performance_TklW', 'Performance_PKwon',
        'Performance_PKcon', 'Performance_OG', 'Performance_Recov',
        'Aerial_Duels_Won', 'Aerial_Duels_Lost', 'Aerial_Duels_Won%'
        ],
    inplace=True
    )

---

In [27]:
processed_df = shooting_df.merge(passing_df,on=['Player', 'Nation', 'Pos', 'Squad', 'League','Age', 'Born', '90s'],how='inner')\
    .merge(pass_types_df,on=['Player', 'Nation', 'Pos', 'Squad', 'League','Age', 'Born', '90s'],how='inner')\
    .merge(gca_df,on=['Player', 'Nation', 'Pos', 'Squad', 'League', 'Age', 'Born', '90s'],how='inner')\
    .merge(defense_df,on=['Player', 'Nation', 'Pos', 'Squad', 'League', 'Age', 'Born', '90s'],how='inner')\
    .merge(possession_df,on=['Player', 'Nation', 'Pos', 'Squad', 'League', 'Age', 'Born', '90s'],how='inner')\
    .merge(misc_df,on=['Player', 'Nation', 'Pos', 'Squad', 'League', 'Age', 'Born', '90s'],how='inner')

    
processed_df.shape

(6158, 100)

In [28]:
processed_df = processed_df.dropna()

In [29]:
processed_df.shape

(6152, 100)

## Merging data sources

### Integrating `Fbref` with `Transfermarkt`

In [30]:
LEAGUES = [
    'Premier League',          
    'Championship',                 
    'Serie A',
    'Serie B',
    'La Liga',
    'La Liga 2',
    'Bundesliga',
    'Bundesliga 2',
    'Ligue 1',
    'Ligue 2',
    'Eredivisie',
    'Belgian Pro League',
    'Primeira Liga',
    'Major League Soccer',
    'Liga MX',
    'Campeonato Brasileiro Série A' 
]

SEASON = 2023
NAME_MATCH_THRESHOLD = 0.9

In [31]:
def normalize_text(text):
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    text = text.replace('-', ' ')
    text = text.replace(' ', '')
    text = text.replace("'",'')
    return text.lower()
    
def find_best_match(row, candidates, threshold=0.9):
    similarities = candidates.apply(lambda x: jaro_winkler(row, x))
    max_sim = similarities.max()
    best_match = candidates[similarities.idxmax()] if max_sim >= threshold else None
    return best_match if isinstance(best_match, str) else None

In [32]:
league= LEAGUES[0]

# 1) Fbref dataset
fbref_data = processed_df.copy()
# 2) Transfermarkt dataset
tm_data = pd.read_csv(f'data/Transfermarkt/players_bio_tm.csv')
tm_data = tm_data[tm_data['Pos']!='Goalkeeper']
tm_data.dropna(inplace=True)

# 3) Get Normalized player name
fbref_data['NormalizedPlayer']  = fbref_data['Player'].apply(lambda text : normalize_text(text))
tm_data['NormalizedPlayer']     = tm_data['Player'].apply(lambda text : normalize_text(text))

fbref_data['matched_key'] = None


for league in LEAGUES:
    # 1) Filter tm data for current league
    tm_data_league = tm_data[tm_data['League']==league].copy()
    # 2) Match Player names
    fbref_data.loc[fbref_data['League'] == league, 'matched_key'] = \
        fbref_data.loc[fbref_data['League'] == league, ['NormalizedPlayer','Born']].apply(
            lambda player: find_best_match(player.values[0], tm_data_league.loc[tm_data_league['BirthYear']==player.values[1],'NormalizedPlayer'], threshold=NAME_MATCH_THRESHOLD), 
            axis=1,
        )

In [33]:
# look for unmatched players
na_df = fbref_data.loc[(fbref_data['League'].isin(LEAGUES)) & (fbref_data['matched_key'].isna()),['Player','League','Squad']]
print(f'Total unmatched players with treshold {NAME_MATCH_THRESHOLD*100}% --> {na_df.shape[0]} out of {len(fbref_data.loc[(fbref_data['League'].isin(LEAGUES))])}')
fbref_data.dropna(subset=['matched_key'],inplace=True)

Total unmatched players with treshold 90.0% --> 505 out of 6152


In [34]:
fbref_data['external_key'] = fbref_data['matched_key'].values + ' ' + fbref_data['League'].values + ' ' + fbref_data['Born'].astype(str).values
tm_data['external_key'] = tm_data['NormalizedPlayer'].values + ' ' + tm_data['League'].values + ' ' + tm_data['BirthYear'].astype(str).values
# Don't consider duplicated keys in tm dataframe
tm_data = tm_data.loc[~tm_data['external_key'].duplicated(keep=False),:]

In [35]:
merged_data = fbref_data.merge(tm_data.loc[:,['external_key','Pos', 'Height', 'Foot', 'MarketValue']],
                              on='external_key',
                              how='inner',
                              suffixes=('_fbref', '_tm'))

In [36]:
merged_data['Height'] = merged_data['Height'].apply(lambda x: int(x.replace(' m', '').replace(',', '')))
merged_data['Foot'] = merged_data['Foot'].apply(lambda x: 1 if x == 'right' else 0)
merged_data.rename(columns={'Foot': 'RightFoot'}, inplace=True)
# Wage STILL TO DO

In [37]:
merged_data.drop(columns=['NormalizedPlayer', 'matched_key', 'external_key'], inplace=True)

In [38]:
merged_data.shape

(5639, 104)

### Integration of `clubelo`

In [39]:
merged_data = merged_data[~ (merged_data['League'].isin(['Major League Soccer','Liga MX','Campeonato Brasileiro Série A'])) ]
merged_data.shape

(4541, 104)

**NOTE**: in this case we did not rely on *Jaro Winkler* distance, we did a manual mapping of the teams.

In [40]:
league_elo = {
    'Premier League'        : {'Country' : 'ENG', 'Level' : 1},          
    'Championship'          : {'Country' : 'ENG', 'Level' : 2},                  
    'Serie A'               : {'Country' : 'ITA', 'Level' : 1},  
    'Serie B'               : {'Country' : 'ITA', 'Level' : 2},  
    'La Liga'               : {'Country' : 'ESP', 'Level' : 1},  
    'La Liga 2'             : {'Country' : 'ESP', 'Level' : 2},  
    'Bundesliga'            : {'Country' : 'GER', 'Level' : 1},  
    'Bundesliga 2'          : {'Country' : 'GER', 'Level' : 2},  
    'Ligue 1'               : {'Country' : 'FRA', 'Level' : 1},  
    'Ligue 2'               : {'Country' : 'FRA', 'Level' : 2},  
    'Eredivisie'            : {'Country' : 'NED', 'Level' : 1},  
    'Belgian Pro League'    : {'Country' : 'BEL', 'Level' : 1},  
    'Primeira Liga'         : {'Country' : 'POR', 'Level' : 1}
}

team_name_mapping = {
    # English Premier League
    'Forest': "Nott'ham Forest",
    'Luton': 'Luton Town',
    'Man City': 'Manchester City',
    'Man United': 'Manchester Utd',
    'Newcastle': 'Newcastle Utd',
    'Sheffield United': 'Sheffield Utd',
    
    # English Championship
    'Birmingham': 'Birmingham City',
    'Cardiff': 'Cardiff City',
    'Coventry': 'Coventry City',
    'Hull': 'Hull City',
    'Ipswich': 'Ipswich Town',
    'Leeds': 'Leeds United',
    'Leicester': 'Leicester City',
    'Norwich': 'Norwich City',
    'Plymouth': 'Plymouth Argyle',
    'Rotherham': 'Rotherham Utd',
    'Stoke': 'Stoke City',
    'Swansea': 'Swansea City',
    
    # Italian Serie A
    'Verona': 'Hellas Verona',
    
    # Italian Serie B
    'Lecco': 'Calcio Lecco 1912',
    'Feralpisalo': 'FeralpiSalò',
    'Suedtirol': 'Südtirol',
    
    # Spanish La Liga
    'Alaves': 'Alavés',
    'Almeria': 'Almería',
    'Atletico': 'Atlético Madrid',
    'Bilbao': 'Athletic Club',
    'Celta': 'Celta Vigo',
    'Cadiz': 'Cádiz',
    'Sociedad': 'Real Sociedad',
    
    # Spanish Segunda División
    'Alcorcon': 'Alcorcón',
    'Andorra CF': 'FC Andorra',
    'Gijon': 'Sporting Gijón',
    'Mirandes': 'CD Mirandés',
    'Ferrol': 'Racing Ferrol',
    'Leganes': 'Leganés',
    'Santander' : 'Racing Sant',
    
    # German Bundesliga
    'Bayern': 'Bayern Munich',
    'Darmstadt': 'Darmstadt 98',
    'Frankfurt': 'Eint Frankfurt',
    'Koeln': 'Köln',
    'Mainz': 'Mainz 05',
    'Werder': 'Werder Bremen',
    
    # German 2. Bundesliga
    'Duesseldorf': 'Düsseldorf',
    'Fuerth': 'Greuther Fürth',
    'Hamburg': 'Hamburger SV',
    'Hannover': 'Hannover 96',
    'Hertha': 'Hertha BSC',
    'Holstein': 'Holstein Kiel',
    'Karlsruhe': 'Karlsruher',
    'Lautern': 'Kaiserslautern',
    'Nuernberg': 'Nürnberg',
    'Osnabrueck': 'Osnabrück',
    'Paderborn': 'Paderborn 07',
    'Rostock': 'Hansa Rostock',
    'Schalke': 'Schalke 04',
    'St Pauli': 'St. Pauli',
    
    # French Ligue 1
    'Clermont': 'Clermont Foot',
    'Paris SG': 'Paris S-G',
    
    # French Ligue 2
    'US Quevilly': 'Quevilly-Rouen',
    'Concarneau': 'US Concarneau',
    'Pau': 'Pau FC',
    'Saint-Etienne': 'Saint-Étienne',
    'Laval' : 'Stade Laval',
    'Rodez' : 'Rodez Aveyron',
    
    # Dutch Eredivisie
    'Alkmaar': 'AZ Alkmaar',
    'Almere': 'Almere City',
    'Go Ahead Eagles': 'Go Ahead Eag',
    'Heracles': 'Heracles Almelo',
    'Nijmegen': 'NEC Nijmegen',
    'PSV': 'PSV Eindhoven',
    'Sittard': 'Fortuna Sittard',
    'Sparta Rotterdam': "Sparta R'dam",
    'Waalwijk': 'RKC Waalwijk',

    # Belgian Pro League
    'Brugge': 'Club Brugge',
    'Leuven': 'OH Leuven',
    'Molenbeek': 'RWD Molenbeek',
    'St Gillis': 'Union SG',
    'St Truiden': 'Sint-Truiden',
    'Standard': 'Standard Liège',

    # Portuguese Primeira Liga
    'Estrela Amadora': 'Estrela',
    'Famalicao': 'Famalicão',
    'Guimaraes': 'Vitória',
    'Sporting': 'Sporting CP',
    'Gil Vicente': 'Gil Vicente FC',
}

In [41]:
elo_df = pd.read_csv('data/Clubelo/clubelo_2024-01-01.csv')

In [42]:
elo_df['Club'] = elo_df['Club'].apply(lambda x : team_name_mapping[x] if x in team_name_mapping else x)

In [43]:
# merged_data['SquadRank']  = merged_data.apply(lambda x : elo_df[elo_df['Club'] == x['Squad']]['Elo'].values[0], axis=1)
# # I want the values merged_data.groupby('League').agg({'SquadRank' : 'mean'}) to be added as new co in merged data
# merged_data['LeagueRank'] = merged_data.groupby('League')['SquadRank'].transform('mean')

In [44]:

league_rank_df = elo_df.groupby(['Country','Level']).agg({'Elo' : 'mean'}).reset_index()
merged_data['LeagueRank'] = merged_data.apply(lambda x : league_rank_df[(league_rank_df['Country'] == league_elo[x['League']]['Country']) & (league_rank_df['Level'] == league_elo[x['League']]['Level'])]['Elo'].values[0], axis=1)
merged_data['SquadRank']  = merged_data.apply(lambda x : elo_df[elo_df['Club'] == x['Squad']]['Elo'].values[0], axis=1)

In [45]:
merged_data.shape

(4541, 106)

Here's the final dataset for our analysis:

In [46]:
merged_data

,Player,Nation,Pos_fbref,Squad,League,Age,Born,90s,Standard_Sh/90,Standard_SoT/90,...,Performance_Crs/90,Performance_Int/90,Aerial_Duels_Won/90,Aerial_Duels_Lost/90,Pos_tm,Height,RightFoot,MarketValue,LeagueRank,SquadRank
0,Max Aarons,eng ENG,DF,Bournemouth,Premier League,23.0,2000.0,13.7,0.15,0.00,...,0.948905,0.583942,0.364964,0.802920,Right-Back,177,1,€16.00m,1772.013940,1711.891602
1,Tosin Adarabioyo,eng ENG,DF,Fulham,Premier League,25.0,1997.0,18.0,0.78,0.28,...,0.055556,1.388889,3.111111,1.555556,Centre-Back,197,1,€20.00m,1772.013940,1717.380493
2,Simon Adingra,ci CIV,FW,Brighton,Premier League,21.0,2002.0,24.7,2.11,0.89,...,3.076923,0.769231,0.323887,0.485830,Left Winger,175,1,€30.00m,1772.013940,1820.636597
3,Nayef Aguerd,ma MAR,DF,West Ham,Premier League,27.0,1996.0,20.6,0.68,0.15,...,0.242718,0.825243,2.330097,1.553398,Centre-Back,190,0,€35.00m,1772.013940,1808.972168
4,Anel Ahmedhodžić,ba BIH,DF,Sheffield Utd,Premier League,24.0,1999.0,29.4,0.65,0.17,...,0.204082,1.156463,1.768707,1.224490,Centre-Back,190,1,€18.00m,1772.013940,1580.956055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536,Bruno Wilson,pt POR,DF,Vizela,Primeira Liga,26.0,1996.0,15.6,1.02,0.26,...,0.320513,1.217949,2.051282,1.794872,Centre-Back,192,1,€800k,1502.914856,1407.074097
4537,Zaydou Youssouf,km COM,MF,Famalicão,Primeira Liga,24.0,1999.0,30.7,1.04,0.33,...,1.270358,0.879479,0.521173,0.456026,Central Midfield,182,0,€7.00m,1502.914856,1473.295410
4538,Rodrigo Zalazar,uy URU,MF,Braga,Primeira Liga,23.0,1999.0,20.2,2.33,0.84,...,6.237624,0.445545,0.990099,1.287129,Central Midfield,178,1,€16.00m,1502.914856,1670.506836
4539,Nermin Zolotić,ba BIH,DF,Casa Pia,Primeira Liga,30.0,1993.0,29.7,0.27,0.10,...,0.134680,0.942761,1.548822,1.245791,Centre-Back,189,0,€600k,1502.914856,1411.265625


In [47]:
merged_data.to_csv('data/training_dataset.csv', index=False)